# Importando as Bibliotecas

In [1]:
import ranking

import pandas as pd
import numpy as np
import string

from scipy import spatial

from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer

import time

# Constantes

In [2]:
VAR_COS = (0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9)

# Lendo os Datasets

In [3]:
arquivos = ['ale_1_1', 'ale_5_1', 'hn_1_1', 'hn_5_1']

lista_df_treino = []
lista_df_teste = []
for arquivo in arquivos:

    df_treino = pd.read_csv(f"Dados/Datasets/Treino/{arquivo}_treino.csv", dtype = {'ean_1': str, 'ean_2': str})
    df_teste = pd.read_csv(f"Dados/Datasets/Teste/{arquivo}_teste.csv", dtype = {'ean_1': str, 'ean_2': str})
    lista_df_treino.append(df_treino)
    lista_df_teste.append(df_teste)

# Aplicando o BoW

## Funções

In [4]:
def calcular_y_pred(limite, distancia):

    y_pred = [1 if num >= limite else 0 for num in distancia]

    return y_pred


def montar_df_resultado(y_teste, y_pred, df_teste, nome):
    df_y = pd.DataFrame(
                        list(zip(
                                 y_teste, y_pred,
                                 df_teste["categoria"].to_list(),
                                 df_teste["titulo_1"].to_list(),
                                 df_teste["titulo_2"].to_list()
                                )
                       ), columns = ['match', 'pred', 'categoria', 'titulo_1', 'titulo_2'])

    return df_y


def salvar_distancia(y_teste, distancia, df_teste, nome):

    df_y = montar_df_resultado(y_teste, distancia, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/Cos/{nome}_distancia.csv', index = False)


def salvar_y_pred(y_teste, y_pred, df_teste, nome, limite):

    df_y = montar_df_resultado(y_teste, y_pred, df_teste, nome)
    df_y.to_csv(f'Dados/Resultados/Cos_{limite}/Resultado/{nome}_y.csv', index = False)


def salvar_relatorio(y_teste, y_pred, nome, tempo, limite):

    relatorio = classification_report(y_teste, y_pred, output_dict = True)
    df_relatorio = pd.DataFrame(relatorio).transpose()
    df_relatorio['modelo'] = nome
    df_relatorio['tempo'] = tempo

    df_relatorio.to_csv(f'Dados/Resultados/Cos_{limite}/Relatório/{nome}_relatório.csv', index = True)

In [5]:
def pipeline_cos(df_teste, df_concat, tam_max):

    cv, titulo_bow = ranking.formatar_entrada_bow(df_concat, mf = tam_max)

    tam_df_concat = len(titulo_bow)/2
    tam_df_teste = df_teste.shape[0]

    if tam_df_concat == tam_df_teste:

        distancia = []
        for i in range( tam_df_teste ):
            distancia.append(ranking.calcular_dis_cos(titulo_bow[i], titulo_bow[i + tam_df_teste]))

        return distancia

    else:
        print ("TAMANHO ERRADO")
        return "ERRO"

## Rodando o BoW

In [6]:
flag_cos = False
if flag_cos == True:

    for nome, df_teste in zip(arquivos, lista_df_teste):

        y_teste = df_teste["match"].to_list()

        # remoção de pontuação e acentos
        ranking.fazer_pre_processamento(df_teste)

        # colocando os titulos em um dataframe com 1 coluna só
        df_concat = ranking.concatenar_titulos(df_teste)

        # calculando o tamanho máximo do título
        tam_max = ranking.calcular_tam_max(df_concat)

        inicio_tempo = time.time()
        distancia = pipeline_cos(df_teste, df_concat, tam_max)
        final_tempo = time.time()

        tempo = final_tempo - inicio_tempo

        salvar_distancia(y_teste, distancia, df_teste, nome)

        for limite in VAR_COS:

            y_pred = calcular_y_pred(limite, distancia)

            salvar_y_pred(y_teste, y_pred, df_teste, nome, ranking.remove_pontuacao(str(limite)) )
            salvar_relatorio(y_teste, y_pred, nome, tempo, ranking.remove_pontuacao(str(limite)) )